In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
from keras.metrics import categorical_accuracy as accuracy
from keras import regularizers
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

In [2]:
import csv
from sklearn.model_selection import cross_val_score

In [5]:
features_train = pd.read_csv('train.csv')
features_test = pd.read_csv('test.csv')
labels_train = features_train['Survived']
features_train.drop('Survived', axis=1, inplace=True)

In [6]:
combined_features = features_train.append(features_test)
combined_features.reset_index(inplace=True)
combined_features.drop('index', axis=1, inplace=True)

In [7]:
combined_features['Title']= combined_features['Name'].map(
    lambda name: name.split(',')[1].split('.')[0].strip())
Title_Dictionary = {
                        "Capt":       "Officer",
                        "Col":        "Officer",
                        "Major":      "Officer",
                        "Jonkheer":   "Royalty",
                        "Don":        "Royalty",
                        "Sir" :       "Royalty",
                        "Dr":         "Officer",
                        "Rev":        "Officer",
                        "the Countess":"Royalty",
                        "Dona":       "Royalty",
                        "Mme":        "Mrs",
                        "Mlle":       "Miss",
                        "Ms":         "Mrs",
                        "Mr" :        "Mr",
                        "Mrs" :       "Mrs",
                        "Miss" :      "Miss",
                        "Master" :    "Master",
                        "Lady" :      "Royalty"

                        }
combined_features['Title']=combined_features.Title.map(Title_Dictionary)

In [8]:
# Processing the ages
grouped_train = combined_features.head(891).groupby(['Sex','Pclass','Title'])
grouped_median_train = grouped_train.median()

In [9]:
grouped_test = combined_features.iloc[891:].groupby(['Sex','Pclass','Title'])
grouped_median_test = grouped_test.median()
def fillAges(row, grouped_median):
    if row['Sex'] == 'female' and row['Pclass'] == 1:
        if row['Title'] == 'Miss':
            return grouped_median.loc['female', 1, 'Miss']['Age']
        elif row['Title'] == 'Mrs':
            return grouped_median.loc['female', 1, 'Mrs']['Age']
        elif row['Title'] == 'Officer':
            return grouped_median.loc['female', 1, 'Officer']['Age']
        elif row['Title'] == 'Royalty':
            return grouped_median.loc['female', 1, 'Royalty']['Age']

    elif row['Sex'] == 'female' and row['Pclass'] == 2:
        if row['Title'] == 'Miss':
            return grouped_median.loc['female', 2, 'Miss']['Age']
        elif row['Title'] == 'Mrs':
            return grouped_median.loc['female', 2, 'Mrs']['Age']

    elif row['Sex'] == 'female' and row['Pclass'] == 3:
        if row['Title'] == 'Miss':
            return grouped_median.loc['female', 3, 'Miss']['Age']
        elif row['Title'] == 'Mrs':
            return grouped_median.loc['female', 3, 'Mrs']['Age']

    elif row['Sex'] == 'male' and row['Pclass'] == 1:
        if row['Title'] == 'Master':
            return grouped_median.loc['male', 1, 'Master']['Age']
        elif row['Title'] == 'Mr':
            return grouped_median.loc['male', 1, 'Mr']['Age']
        elif row['Title'] == 'Officer':
            return grouped_median.loc['male', 1, 'Officer']['Age']
        elif row['Title'] == 'Royalty':
            return grouped_median.loc['male', 1, 'Royalty']['Age']

    elif row['Sex'] == 'male' and row['Pclass'] == 2:
        if row['Title'] == 'Master':
            return grouped_median.loc['male', 2, 'Master']['Age']
        elif row['Title'] == 'Mr':
            return grouped_median.loc['male', 2, 'Mr']['Age']
        elif row['Title'] == 'Officer':
            return grouped_median.loc['male', 2, 'Officer']['Age']

    elif row['Sex'] == 'male' and row['Pclass'] == 3:
        if row['Title'] == 'Master':
            return grouped_median.loc['male', 3, 'Master']['Age']
        elif row['Title'] == 'Mr':
            return grouped_median.loc['male', 3, 'Mr']['Age']

combined_features.head(891).Age = combined_features.head(891).apply(lambda r: fillAges(r, grouped_median_train) if np.isnan(r['Age'])
    else r['Age'], axis=1)

combined_features.iloc[891:].Age = combined_features.iloc[891:].apply(lambda r: fillAges(r, grouped_median_test) if np.isnan(r['Age'])
    else r['Age'], axis=1)



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
# Processing names: drop names and encode the titles using dummy encoding from pandas
combined_features.drop('Name', axis=1, inplace=True)
dummy_titles = pd.get_dummies(combined_features['Title'], prefix='Title')
combined_features = pd.concat([combined_features, dummy_titles], axis=1)
combined_features.drop('Title', axis=1, inplace=True)

In [11]:
# Processing Fares: fill empty spaces with mean
combined_features.head(891).Fare.fillna(combined_features.head(891).Fare.mean(), inplace=True)
combined_features.iloc[891:].Fare.fillna(combined_features.iloc[891:].Fare.mean(), inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [12]:
#Processing Embarked: drop it
combined_features.drop('Embarked', axis=1, inplace=True)

#Processing Ticket: drop it
combined_features.drop('Ticket', axis=1, inplace=True)

In [13]:
#Processing Cabin:
combined_features.Cabin.fillna('U', inplace=True)

# mapping each Cabin value with the cabin letter
combined_features['Cabin'] = combined_features['Cabin'].map(lambda c: c[0])

cabin_dummies = pd.get_dummies(combined_features['Cabin'], prefix='Cabin')
combined_features = pd.concat([combined_features, cabin_dummies], axis=1)
combined_features.drop('Cabin', axis=1, inplace=True)

In [14]:
# Processing sex
combined_features['Sex'] = combined_features['Sex'].map({'male':1,'female':0})

In [15]:
# Processing pclass
pclass_dummies = pd.get_dummies(combined_features['Pclass'], prefix="Pclass")
combined_features = pd.concat([combined_features, pclass_dummies], axis=1)
combined_features.drop('Pclass', axis=1, inplace=True)

In [16]:
# Processing PassengerId
combined_features.drop('PassengerId', axis=1, inplace=True)

In [17]:
# Processing family
combined_features['FamilySize'] = combined_features['Parch'] + combined_features['SibSp'] + 1
combined_features['Singleton'] = combined_features['FamilySize'].map(lambda s: 1 if s == 1 else 0)
combined_features['SmallFamily'] = combined_features['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
combined_features['LargeFamily'] = combined_features['FamilySize'].map(lambda s: 1 if 5 <= s else 0)
combined_features.drop('Parch', axis=1, inplace=True)
combined_features.drop('SibSp', axis=1, inplace=True)

In [18]:
#Scaling
scale=max(combined_features['Age'])
combined_features['Age']/=scale
mean=np.mean(combined_features['Age'])
combined_features['Age']-=mean

scale=max(combined_features['Fare'])
combined_features['Fare']/=scale
mean=np.mean(combined_features['Fare'])
combined_features['Fare']-=mean


print(combined_features.head())

   Sex       Age      Fare  Title_Master  Title_Miss  Title_Mr  Title_Mrs  \
0    1 -0.090295 -0.050841             0           0         1          0   
1    0  0.109705  0.074144             0           0         0          1   
2    0 -0.040295 -0.049523             0           1         0          0   
3    0  0.072205  0.038652             0           0         0          1   
4    1  0.072205 -0.049279             0           0         1          0   

   Title_Officer  Title_Royalty  Cabin_A  ...  Cabin_G  Cabin_T  Cabin_U  \
0              0              0        0  ...        0        0        1   
1              0              0        0  ...        0        0        0   
2              0              0        0  ...        0        0        1   
3              0              0        0  ...        0        0        0   
4              0              0        0  ...        0        0        1   

   Pclass_1  Pclass_2  Pclass_3  FamilySize  Singleton  SmallFamily  \
0        

In [19]:
features_train=combined_features[:891]
features_test=combined_features[891:]

In [20]:
model = Sequential()
model.add(Dense(512, input_dim=25))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
labels_train = np_utils.to_categorical(labels_train)

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
clf=model

W0707 23:44:19.985840 4501956032 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 23:44:20.015707 4501956032 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 23:44:20.025672 4501956032 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0707 23:44:20.051149 4501956032 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:13

In [21]:
clf.fit(np.array(features_train),labels_train,validation_split=0.1,epochs=88)
out = clf.predict(np.array(features_test))
#print(accuracy_score(out,labels_test))
pred=[]
for x in out:
    if(x[1]>0.5):
        pred.append(1)
    else:
        pred.append(0)

out1=pred

W0707 23:44:55.658179 4501956032 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 801 samples, validate on 90 samples
Epoch 1/88
801/801 [==============================] - 1s 784us/step - loss: 0.5591 - acc: 0.7241 - val_loss: 0.4189 - val_acc: 0.8333
Epoch 2/88
801/801 [==============================] - 0s 123us/step - loss: 0.4569 - acc: 0.8127 - val_loss: 0.3838 - val_acc: 0.8444
Epoch 3/88
801/801 [==============================] - 0s 135us/step - loss: 0.4211 - acc: 0.8127 - val_loss: 0.3557 - val_acc: 0.8556
Epoch 4/88
801/801 [==============================] - 0s 187us/step - loss: 0.4127 - acc: 0.8302 - val_loss: 0.3385 - val_acc: 0.8556
Epoch 5/88
801/801 [==============================] - 0s 175us/step - loss: 0.4257 - acc: 0.8215 - val_loss: 0.3621 - val_acc: 0.8556
Epoch 6/88
801/801 [==============================] - 0s 141us/step - loss: 0.4127 - acc: 0.8315 - val_loss: 0.3435 - val_acc: 0.8444
Epoch 7/88
801/801 [==============================] - 0s 150us/step - loss: 0.4009 - acc: 0.8277 - val_loss: 0.3372 - val_acc: 0.8444
Epoch 8/88
801/80

801/801 [==============================] - 0s 122us/step - loss: 0.3455 - acc: 0.8577 - val_loss: 0.3255 - val_acc: 0.8556
Epoch 62/88
801/801 [==============================] - 0s 116us/step - loss: 0.3640 - acc: 0.8427 - val_loss: 0.3196 - val_acc: 0.8333
Epoch 63/88
801/801 [==============================] - 0s 114us/step - loss: 0.3342 - acc: 0.8689 - val_loss: 0.3357 - val_acc: 0.8333
Epoch 64/88
801/801 [==============================] - 0s 112us/step - loss: 0.3470 - acc: 0.8602 - val_loss: 0.3090 - val_acc: 0.8333
Epoch 65/88
801/801 [==============================] - 0s 113us/step - loss: 0.3387 - acc: 0.8502 - val_loss: 0.3178 - val_acc: 0.8333
Epoch 66/88
801/801 [==============================] - 0s 116us/step - loss: 0.3437 - acc: 0.8627 - val_loss: 0.3480 - val_acc: 0.8444
Epoch 67/88
801/801 [==============================] - 0s 115us/step - loss: 0.3392 - acc: 0.8639 - val_loss: 0.3081 - val_acc: 0.8667
Epoch 68/88
801/801 [==============================] - 0s 114us/ste

In [23]:
idk = open('test.csv','r')
idk = csv.DictReader(idk)
pred = []
for row in idk:
    pred.append(row["PassengerId"])
est = csv.writer(open('result.csv', 'w'), delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
est.writerows([["PassengerID","Survived"]])
k=0
for x in pred:
    est.writerows([[str(x), str(out1[k])]])
    k+=1